In [ ]:
import os
import ccxt
import pandas as pd
import csv
from datetime import datetime
import time
# import pytz
import json
# import odo 

print('python', sys.version)
print('CCXT Version:', ccxt.__version__)

In [ ]:
from pymongo import MongoClient
from bson.objectid import ObjectId
from pprint import pprint

client = MongoClient()
db=client.data_feeder


In [ ]:
exchanges = {}
config = False
with open ('config.json') as js: 
        config = json.load(js)

for exkey, exconf in config['exchanges'].items():
        if exconf['enabled'] and not exkey in exchanges:
                excred = exconf['cred']
                for crk, crv in excred.items():
                        if crv[0] == "$" : 
                                excred[crk] = os.getenv(crv[1:])

                exchanges[exkey] = getattr(ccxt, exkey) (excred)

In [ ]:
# exchanges
exchange_id = 'kucoinfutures'
exchange = exchanges[exchange_id]
exchange

In [ ]:
def retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    num_retries = 0
    try:
        num_retries += 1
        ohlcv = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
        # print('Fetched', len(ohlcv), symbol, 'candles from', exchange.iso8601 (ohlcv[0][0]), 'to', exchange.iso8601 (ohlcv[-1][0]))
        return ohlcv
    except Exception:
        if num_retries > max_retries:
            raise  # Exception('Failed to fetch', timeframe, symbol, 'OHLCV in', max_retries, 'attempts')


def scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit):
    timeframe_duration_in_seconds = exchange.parse_timeframe(timeframe)
    timeframe_duration_in_ms = timeframe_duration_in_seconds * 1000
    timedelta = limit * timeframe_duration_in_ms
    now = exchange.milliseconds()
    all_ohlcv = []
    fetch_since = since
    while fetch_since < now:
        ohlcv = retry_fetch_ohlcv(exchange, max_retries, symbol, timeframe, fetch_since, limit)
        fetch_since = (ohlcv[-1][0] + 1) if len(ohlcv) else (fetch_since + timedelta)
        all_ohlcv = all_ohlcv + ohlcv
        if len(all_ohlcv):
            print(len(all_ohlcv), 'candles in total from', exchange.iso8601(all_ohlcv[0][0]), 'to', exchange.iso8601(all_ohlcv[-1][0]))
        else:
            print(len(all_ohlcv), 'candles in total from', exchange.iso8601(fetch_since))
    return exchange.filter_by_since_limit(all_ohlcv, since, None, key=0)


def write_to_csv(filename, data):
    with open(filename, mode='w') as output_file:
        csv_writer = csv.writer(output_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerows(data)


def scrape_candles_to_csv(filename, exchange_id, max_retries, symbol, timeframe, since, limit):
    # instantiate the exchange by id
    exchange = getattr(ccxt, exchange_id)({
        'enableRateLimit': True,  # required by the Manual
    })
    # convert since from string to milliseconds integer if needed
    if isinstance(since, str):
        since = exchange.parse8601(since)
    # preload all markets from the exchange
    exchange.load_markets()
    # fetch all candles
    ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
    # save them to csv file
    write_to_csv(filename, ohlcv)
    print('Saved', len(ohlcv), 'candles from', exchange.iso8601(ohlcv[0][0]), 'to', exchange.iso8601(ohlcv[-1][0]), 'to', filename)


# -----------------------------------------------------------------------------
# Binance's BTC/USDT candles start on 2017-08-17
# scrape_candles_to_csv('kucionfutures.csv', 'kucoinfutures', 3, 'XBTUSDTM', '1d', '2021-01-01T00:00:00Z', 100)

In [ ]:
data_dir = f'./data/{exchange_id}'
if not os.path.exists(data_dir): os.makedirs(data_dir) 

In [ ]:
symbol = 'ALGOUSDTM'
timeframe = '1d'
since = '2020-01-01T00:00:00Z'
limit = 500
max_retries = 3
if isinstance(since, str): since = exchange.parse8601(since)

csv_fname = f'{data_dir}/{symbol}-{timeframe}.csv'

# ohlcv = scrape_ohlcv(exchange, max_retries, symbol, timeframe, since, limit)
# write_to_csv(csv_fname, ohlcv)

scrape_candles_to_csv(csv_fname, exchange_id, max_retries, symbol, timeframe, since, limit)

# ohlcv
# klines_json = exchange.fetch_ohlcv (symbol, timeframe,params={'startAt': since_date.timestamp() * 1000,'endAt': to_date.timestamp() * 1000, 'maxCount': 1000000})
# df = pd.DataFrame(klines_json)
# df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
# df['date'] = pd.to_datetime(df['timestamp'], unit='ms', utc=False)
# df.set_index(pd.DatetimeIndex(df["date"]), inplace=True, drop=True)
# df

In [ ]:
# symbol = 'ALGOUSDTM'
# timeframe = '15m'

symbols = ['ALGOUSDTM']
timeframes = ['5m', '15m', '1h', '8h', '1d']

for symbol in symbols:
  for timeframe in timeframes:
    # print(timeframe, symbol)

    csv_fname = f'{data_dir}/{symbol}-{timeframe}.csv'
    # df = pd.DataFrame(ohlcv)
    df = pd.read_csv(csv_fname)

    df.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']
    df['date'] = pd.to_datetime(df['timestamp'], unit='ms', utc=False)
    # df.set_index(df["timestamp"], inplace=True, drop=True)
    # df.drop(columns=['timestamp'], inplace=True)
    df['exchange_id'] = exchange_id
    df['symbol'] = symbol
    df['timeframe'] = timeframe

    df = df[['timestamp', 'date', 'exchange_id', 'symbol', 'timeframe', 'open', 'high', 'low', 'close', 'volume']]
    db.klines.insert_many(df.to_dict("records"))

# # df.reset_index(inplace=True)
# df

In [ ]:
exchange

In [ ]:

since = '2020-01-01T00:00:00Z'
if isinstance(since, str): since = exchange.parse8601(since)
since
